<a href="https://colab.research.google.com/github/sanaeaba/Recommende_system_coursera/blob/master/FM_Coursera.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastFM

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastFM: filename=fastFM-0.2.10-cp310-cp310-linux_x86_64.whl size=591408 sha256=4ab0a80b5926e8d68b4f4975f7ae38ed840df5d9e42c329808a3c76b28d5cf22
  Stored in directory: /root/.cache/pip/wheels/93/92/52/2da7997fcb7a7ce9042ff3b33836ef0c2fd47aa95382d7a113
Successfully built fastFM


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import mean_squared_error, mean_absolute_error
from fastFM import als
from copy import deepcopy

# Load the Coursera dataset (replace 'merged_coursera_dataset.csv' with the actual file path)
data = pd.read_csv('merged_coursera_datase.csv')

# Create feature vectors for user, course, Sentiment, and course_difficulty
v = DictVectorizer()
X = v.fit_transform(data[['user_id', 'course_id', 'Sentiment', 'course_difficulty']].to_dict(orient='records'))
y = data['rating'].values

# Split the data into training, validation, and test sets
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Set hyperparameters
batch_size = 512  # Define your desired batch size

# Create and train a Factorization Machines model with batched training
num_batches = X_train.shape[0] // batch_size
fm = als.FMRegression(n_iter=40, l2_reg=0.01, init_stdev=0.01, rank=2)

best_rmse = float('inf')
best_mae = float('inf')
best_fm_model = None

for i in range(num_batches):
    start = i * batch_size
    end = (i + 1) * batch_size
    X_batch = X_train[start:end]
    y_batch = y_train[start:end]
    fm.fit(X_batch, y_batch)

    # Periodically evaluate the model on the validation set
    if i % 10 == 0:
        y_val_pred = fm.predict(X_val)
        val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
        val_mae = mean_absolute_error(y_val, y_val_pred)

        if val_rmse < best_rmse:
            best_rmse = val_rmse
            best_mae = val_mae
            best_fm_model = deepcopy(fm)

# Use the best model on the test set
y_test_pred = best_fm_model.predict(X_test)

# Calculate RMSE and MAE on the test set
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

print(f"RMSE on Test Set using Best Model: {test_rmse:.4f}")
print(f"MAE on Test Set using Best Model: {test_mae:.4f}")


RMSE on Test Set using Best Model: 1.8657
MAE on Test Set using Best Model: 1.3533
